In [146]:
import xarray as xr

In [147]:
TIME_START = "2002-01-31"
TIME_END = "2014-11-30"

## HOT


In [148]:
cmems_hot = xr.load_dataset(
    "../../../1_data_processing/1_1_Forcing/products/Hot_cmems_climato_2002_2014.zarr", engine="zarr"
)
cmems_hot

<xarray.Dataset> Size: 825kB
Dimensions:              (latitude: 1, longitude: 1, depth: 3, time: 4687)
Coordinates:
  * depth                (depth) int64 24B 0 1 2
  * latitude             (latitude) float64 8B 22.75
  * longitude            (longitude) int64 8B -158
  * time                 (time) datetime64[ns] 37kB 2002-01-31 ... 2014-11-30
Data variables: (12/13)
    T                    (latitude, longitude, depth, time) float64 112kB 23....
    U                    (latitude, longitude, depth, time) float64 112kB -0....
    V                    (latitude, longitude, depth, time) float64 112kB -0....
    mnkc_epi             (latitude, longitude, time) float64 37kB 0.3072 ... ...
    mnkc_hmlmeso         (latitude, longitude, time) float64 37kB 0.8849 ... ...
    mnkc_lmeso           (latitude, longitude, time) float64 37kB 2.279 ... 2...
    ...                   ...
    mnkc_mumeso          (latitude, longitude, time) float64 37kB 0.6114 ... ...
    mnkc_umeso           (latitude, longitude, time) float64 37kB 0.5152 ... ...
    npp                  (latitude, longitude, time) float64 37kB 331.4 ... 2...
    pelagic_layer_depth  (latitude, longitude, depth, time) float64 112kB 115...
    zeu                  (latitude, longitude, time) float64 37kB 76.89 ... 7...
    zooc                 (latitude, longitude, time) float64 37kB 0.2935 ... ...

In [149]:
zoo_obs_hot = xr.load_dataset(
    "../../../1_data_processing/1_1_Forcing/products/Hot_obs_zoo_climato_monthly_2002_2015.zarr", engine="zarr"
)
zoo_obs_hot

<xarray.Dataset> Size: 6kB
Dimensions:            (time: 155, latitude: 1, longitude: 1, layer: 1)
Coordinates:
  * latitude           (latitude) float64 8B 22.75
  * layer              (layer) int64 8B 0
  * longitude          (longitude) int64 8B -158
  * time               (time) datetime64[ns] 1kB 2002-01-31 ... 2014-11-30
Data variables:
    day                (time, latitude, longitude, layer) float64 1kB 1.397 ....
    day_lowess_0.15    (time, latitude, longitude, layer) float64 1kB 1.465 ....
    night              (time, latitude, longitude, layer) float64 1kB 2.237 ....
    night_lowess_0.15  (time, latitude, longitude, layer) float64 1kB 2.538 ....
Attributes:
    desc:     Aggregated and smoothed observations using lowess filtre.

In [150]:
npp_obs_hot = xr.load_dataset(
    "../../../1_data_processing/1_1_Forcing/products/Hot_observed_npp_climato.zarr", engine="zarr"
)
npp_obs_hot

<xarray.Dataset> Size: 36kB
Dimensions:          (time: 1489, latitude: 1, longitude: 1)
Coordinates:
  * latitude         (latitude) float64 8B 22.75
  * longitude        (longitude) int64 8B -158
  * time             (time) datetime64[ns] 12kB 1994-02-20 ... 2022-08-28
Data variables:
    l12              (time, latitude, longitude) float64 12kB 296.8 ... 410.9
    l12_lowess_0.05  (time, latitude, longitude) float64 12kB 298.8 ... 408.5
Attributes:
    desc:     L12 multiplied by ZEU (from CMEMS).
    type:     climatology

# Barents sea


In [151]:
cmems_barents = xr.load_dataset(
    "../../../1_data_processing/1_1_Forcing/1_optional_extract_barents_sea/Barents_sea_forcing_2_depth.zarr",
    engine="zarr",
)
cmems_barents

<xarray.Dataset> Size: 468kB
Dimensions:              (latitude: 1, longitude: 1, time: 7305, depth: 2)
Coordinates:
  * depth                (depth) int64 16B 0 1
  * latitude             (latitude) int64 8B 72
  * longitude            (longitude) int64 8B 20
  * time                 (time) datetime64[ns] 58kB 1998-01-01 ... 2017-12-31
Data variables:
    T                    (latitude, longitude, time, depth) float64 117kB 5.2...
    npp                  (latitude, longitude, time) float64 58kB 3.654e-06 ....
    pelagic_layer_depth  (latitude, longitude, time, depth) float64 117kB 150...
    zeu                  (latitude, longitude, time) float64 58kB 100.2 ... 1...
    zooc                 (latitude, longitude, time) float64 58kB 0.8288 ... ...
Attributes:
    Conventions:  CF-1.7
    history:      Created on 2024-10-17
    institution:  CLS
    references:   http://www.cls.fr; http://www.seapodym.eu
    source:       SEAPODYM-LMTL 3.0.0
    title:        Global ocean low and mid trophic levels biomass content hin...

In [152]:
zoo_obs_barents = xr.load_dataset(
    "../../../1_data_processing/1_1_Forcing/1_optional_extract_barents_sea/zoo_2000_2017.zarr", engine="zarr"
)
zoo_obs_barents

<xarray.Dataset> Size: 3kB
Dimensions:    (latitude: 1, layer: 1, longitude: 1, time: 216)
Coordinates:
  * latitude   (latitude) int64 8B 72
  * layer      (layer) int64 8B 0
  * longitude  (longitude) int64 8B 20
  * time       (time) datetime64[ns] 2kB 2000-01-01 2000-02-01 ... 2017-12-01
Data variables:
    night      (time, latitude, longitude, layer) float64 2kB 0.616 ... 0.936

## Gather


### Primary production


In [153]:
primary_production_hot = (
    npp_obs_hot["l12"]
    .rename("primary_production")
    .resample(time="1D")
    .interpolate("linear")
    .sel(time=slice(TIME_START, TIME_END))
).transpose("time", "latitude", "longitude")

primary_production_barents = (
    cmems_barents["npp"]
    .rename("primary_production")
    .resample(time="1D")
    .interpolate("linear")
    .sel(time=slice(TIME_START, TIME_END))
).transpose("time", "latitude", "longitude")

gathered_primary_production = xr.concat([primary_production_hot, primary_production_barents], dim="latitude")
gathered_primary_production

<xarray.DataArray 'primary_production' (time: 4687, latitude: 2, longitude: 2)> Size: 150kB
array([[[2.72055903e+02,            nan],
        [           nan, 3.22143971e-01]],

       [[2.72197788e+02,            nan],
        [           nan, 5.70163508e-01]],

       [[2.72339673e+02,            nan],
        [           nan, 6.69941482e-01]],

       ...,

       [[2.77754534e+02,            nan],
        [           nan, 3.65403412e-06]],

       [[2.76520355e+02,            nan],
        [           nan, 3.65403412e-06]],

       [[2.75286176e+02,            nan],
        [           nan, 3.65403412e-06]]])
Coordinates:
  * longitude  (longitude) int64 16B -158 20
  * time       (time) datetime64[ns] 37kB 2002-01-31 2002-02-01 ... 2014-11-30
  * latitude   (latitude) float64 16B 22.75 72.0
Attributes:
    desc:     L12 multiplied by ZEU (from CMEMS)
    station:  hot
    type:     in situ
    units:    mg m-2 day-1

### Pelagic layer


In [186]:
pelagic_layer_hot = (
    cmems_hot["pelagic_layer_depth"].resample(time="1D").interpolate("linear").sel(time=slice(TIME_START, TIME_END))
).transpose("time", "latitude", "longitude", "depth")

pelagic_layer_barents = (
    cmems_barents["pelagic_layer_depth"]
    .rename("pelagic_layer_depth")
    .resample(time="1D")
    .interpolate("linear")
    .sel(time=slice(TIME_START, TIME_END))
).transpose("time", "latitude", "longitude", "depth")

gathered_pelagic_layer = xr.concat([pelagic_layer_hot, pelagic_layer_barents], dim="latitude")
gathered_pelagic_layer

<xarray.DataArray 'pelagic_layer_depth' (time: 4687, latitude: 2, longitude: 2,
                                         depth: 3)> Size: 450kB
array([[[[115.3424172 , 346.02587724, 807.39408241],
         [         nan,          nan,          nan]],

        [[         nan,          nan,          nan],
         [160.0854944 , 480.25887233,          nan]]],


       [[[115.06894863, 345.20405792, 805.4767182 ],
         [         nan,          nan,          nan]],

        [[         nan,          nan,          nan],
         [160.17731914, 480.53166164,          nan]]],


       [[[114.9236042 , 344.76828165, 804.45892165],
         [         nan,          nan,          nan]],

        [[         nan,          nan,          nan],
         [160.22743791, 480.68353941,          nan]]],

...

       [[[116.908693  , 350.72393359, 818.3560854 ],
         [         nan,          nan,          nan]],

        [[         nan,          nan,          nan],
         [137.08312709, 411.22545806,          nan]]],


       [[[115.42080809, 346.25963632, 807.94011998],
         [         nan,          nan,          nan]],

        [[         nan,          nan,          nan],
         [138.05936912, 414.15355768,          nan]]],


       [[[116.04343742, 348.12855239, 812.29852529],
         [         nan,          nan,          nan]],

        [[         nan,          nan,          nan],
         [138.98943029, 416.94902155,          nan]]]])
Coordinates:
  * depth      (depth) int64 24B 0 1 2
  * longitude  (longitude) int64 16B -158 20
  * time       (time) datetime64[ns] 37kB 2002-01-31 2002-02-01 ... 2014-11-30
  * latitude   (latitude) float64 16B 22.75 72.0
Attributes:
    standard_name:  sea_water_pelagic_layer_bottom_depth
    units:          m

### Temperature


In [175]:
temperature_hot = (
    cmems_hot["T"].rename("temperature").resample(time="1D").interpolate("linear").sel(time=slice(TIME_START, TIME_END))
).transpose("time", "latitude", "longitude", "depth")

temperature_barents = (
    cmems_barents["T"]
    .rename("temperature")
    .resample(time="1D")
    .interpolate("linear")
    .sel(time=slice(TIME_START, TIME_END))
).transpose("time", "latitude", "longitude", "depth")

gathered_temperature = xr.concat([temperature_hot, temperature_barents], dim="latitude")
gathered_temperature

<xarray.DataArray 'temperature' (time: 4687, latitude: 2, longitude: 2, depth: 3)> Size: 450kB
array([[[[23.23587628, 16.25128179,  6.63389493],
         [        nan,         nan,         nan]],

        [[        nan,         nan,         nan],
         [ 4.81129912,  4.90338062,         nan]]],


       [[[23.22053677, 16.2553568 ,  6.63674259],
         [        nan,         nan,         nan]],

        [[        nan,         nan,         nan],
         [ 4.79262268,  4.88867393,         nan]]],


       [[[23.20255336, 16.24173462,  6.65982947],
         [        nan,         nan,         nan]],

        [[        nan,         nan,         nan],
         [ 4.77687203,  4.88738334,         nan]]],

...

       [[[24.54037003, 16.29319134,  6.53758882],
         [        nan,         nan,         nan]],

        [[        nan,         nan,         nan],
         [ 6.51678149,  5.81795204,         nan]]],


       [[[24.55222637, 16.38269122,  6.64419404],
         [        nan,         nan,         nan]],

        [[        nan,         nan,         nan],
         [ 6.5006592 ,  5.80981657,         nan]]],


       [[[24.52805767, 16.28962571,  6.59112186],
         [        nan,         nan,         nan]],

        [[        nan,         nan,         nan],
         [ 6.47896912,  5.79901545,         nan]]]])
Coordinates:
  * depth      (depth) int64 24B 0 1 2
  * longitude  (longitude) int64 16B -158 20
  * time       (time) datetime64[ns] 37kB 2002-01-31 2002-02-01 ... 2014-11-30
  * latitude   (latitude) float64 16B 22.75 72.0
Attributes:
    standard_name:  sea_water_potential_temperature_vertical_mean_over_pelagi...
    units:          degree_Celsius

## Export


In [189]:
xr.Dataset(
    {
        "temperature": gathered_temperature,
        "primary_production": gathered_primary_production,
        "pelagic_layer_depth": gathered_pelagic_layer,
    }
).to_zarr("./forcing.zarr")